# Bioimage Analysis Workflow 2 - Seed Clustering (1)

## Loading and Handling Image Data

In [ ]:
filepath = r'example_data/seed_img_1.JPG'

from skimage.io import imread

img = imread(filepath)

print("Loaded array has shape:", img.shape)
print("Loaded values are of type:", img.dtype)

In [ ]:
# Split channel
img_ch0 = img[:,:,0]
img_ch1 = img[:,:,1]
img_ch2 = img[:,:,2]

### Show image from different channels

In [ ]:
import matplotlib.pyplot as plt

# Define a range to visulize
x_start = 1000
x_end = 2000
y_start = 1000
y_end = 2000

plt.figure(figsize=(15,15))
plt.subplot(131)
plt.imshow(img_ch0[y_start: y_end, x_start: x_end], cmap = 'gray', interpolation='none')
plt.title('ch_0')
plt.subplot(132)
plt.imshow(img_ch1[y_start: y_end, x_start: x_end], cmap = 'gray', interpolation='none')
plt.title('ch_1')
plt.subplot(133)
plt.imshow(img_ch2[y_start: y_end, x_start: x_end], cmap = 'gray', interpolation='none')
plt.title('ch_2')

It can be seen that channel 2 is strong for all seeds

## Segment Seeds by Otsu Thresholding

In [ ]:
# Calculate Otsu's threshold by threshold_otsu from `skimage.filters.thresholding`
### Your code here!
from skimage.filters.thresholding import threshold_otsu

# Calculate threshold
### Your code here!
thresh = threshold_otsu(img_ch2)

In [ ]:
# Perform Global thresholding with the calculated threshold
### Your code here!
seeds_seg = img_ch2 < thresh

# Show the segmented image
### Your code here!
plt.imshow(seeds_seg[y_start: y_end, x_start: x_end], cmap = 'gray')
plt.title('ch_2')

## Distance and Watershed transform

### Distance transform

In [ ]:
# (i) Run a distance transform on the seeds seg
# The image processing module scipy.ndimage as ndi
### Your code here!
import scipy.ndimage as ndi

# Use the function 'ndi.distance_transform_edt' for distance transform
### Your code here!
dist_trans = ndi.distance_transform_edt(seeds_seg)

In [ ]:
# (ii) Visualize the output and understand what you are seeing.
### Your code here!
plt.imshow(dist_trans[y_start: y_end, x_start: x_end])

In [ ]:
# (iii) Smoothen the distance transform
# Use `ndi.gaussian_filter`
### Your code here!
dist_trans_smooth = ndi.gaussian_filter(dist_trans, sigma=3)

# Show result
### Your code here!
plt.imshow(dist_trans_smooth[y_start: y_end, x_start: x_end])

In [ ]:
# (iv) Get the local maxima (the 'peaks') from the distance transform

# Use the function 'peak_local_max' from the module 'skimage.feature'. This function will return the
# indices/ coordinates of the pixels where the local maxima are. 

from skimage.feature import peak_local_max

seeds = peak_local_max(dist_trans_smooth, min_distance=8)

In [ ]:
# (v) However, we instead need a boolean mask of the same shape as the original image, where all 
# the local maximum pixels are labeled as `1` and everything else as `0`.

# Let's do it step by step. First try have a look at what is in seeds. Can you get these values?
# Number of seeds
print(f'There are {np.shape(seeds)[0]} seeds')
# The X coordinate of the first seed
print(seeds[0][0])
# The Y coordinate of the 13th seed
print(seeds[13][1])

In [ ]:
# Now, we will start by creating a boolean matrix/ image same size as the original image, but with
# all pixel values as 0/ false
seeds_mask = np.zeros_like(dist_trans_smooth, dtype = bool)

# For loop through all entries in seeds
for seed_id in range(np.shape(seeds)[0]):
    seeds_mask[seeds[seed_id][0],seeds[seed_id][1]] = 1

# P.S. for advanced Python coder - this also works without a for loop:
# seeds_mask[tuple(seeds.T)] = True

In [ ]:
# (vi) Visualize the output 
# Expand seeds mask and show results
from skimage.morphology import disk

seeds_dil = ndi.binary_dilation(seeds_mask, disk(2))

plt.figure(figsize=(5,5))
plt.imshow(seeds_seg[y_start: y_end, x_start: x_end], cmap = 'gray')
plt.imshow(seeds_dil[y_start: y_end, x_start: x_end], interpolation='none', alpha = 0.5)

In [ ]:
# (vii) Label the seeds
seeds_labeled = ndi.label(seeds_dil)[0]

# Visualize
seeds_labeled_mask = np.ma.array(seeds_labeled, mask=seeds_labeled==0)
plt.imshow(seeds_seg[y_start: y_end, x_start: x_end], interpolation='none', cmap='gray')
plt.imshow(seeds_labeled_mask[y_start: y_end, x_start: x_end], interpolation='none', cmap='prism')

### Watershed Transform

In [ ]:
# (i) Perform watershed
from skimage.segmentation import watershed

seed_rois = watershed(-dist_trans, markers=seeds_labeled, mask = seeds_seg)

seeds_seed_rois_mask = np.ma.array(seed_rois, mask=seed_rois==0)
plt.imshow(img_ch1[y_start: y_end, x_start: x_end], interpolation='none', cmap='gray')
plt.imshow(seeds_seed_rois_mask[y_start: y_end, x_start: x_end], interpolation='none', cmap='prism', alpha = 0.3)

## Save result for later processing

In [ ]:
# Import imsave from skimage.io
from skimage.io import imsave

# Save image
imsave(r"example_data/seed_labels.tif", seed_rois.astype(np.uint16))